In [ ]:
!pip install selenium

In [4]:
#Webdriver de Selenium qui permet de contrôler un navigateur
from selenium import webdriver

#Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.by import By

#Pour attendre qu'une condition soit remplie avant de poursuivre l'exécution du script
from selenium.webdriver.support.ui import WebDriverWait

#fournit des conditions d'attente prédéfinies pour être utilisées avec WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select


#Pour utiliser les fonctionnalités liées à la gestion du temps
import time

import re

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

import pandas as pd

from datetime import datetime

from pprint import pprint 


from bs4 import BeautifulSoup

<h1> Columns generation

In [5]:
def twitter_time(soup1):
    times = soup1.find_all('time')
    dates=[]
    for time in times:
        dates.append(time.get('datetime'))
    return dates

In [6]:
def twitter_user(soup1):
    users = soup1.find_all('div',attrs={'data-testid':'User-Name'})

    username=[]
    for div in users:
        divss = div.find('a')
        print(divss['href'])
        username.append(divss.get('href')[1:])
    return username

In [7]:
def twitter_tweets(soup1):
    tweets = soup1.find_all('div',attrs={'data-testid':"tweetText"})

    texts=[]
    for tweet in tweets:
        texts.append(tweet.text)
    return texts

In [8]:
def twitter_react(soup1):
    likes = soup1.find_all('div',attrs={'role':'group'})

    reacts=[]
    order = ['replies', 'reposts', 'likes', 'bookmarks', 'views']
    
    for like in likes:
        react_text=like['aria-label']

        # Initialize dictionary to store numbers
        react_stat = {word: 0 for word in order}

        # Find all numbers and words using regex
        matches = re.findall(r"(\d+) (\w+)", react_text)

        # Update numbers dictionary with extracted values
        for match in matches:
            number, word = match
            if word in react_stat:
                react_stat[word] = int(number)
        reacts.append([react_stat[word] for word in order])

        
    react_sorted = [[row[i] for row in reacts] for i in range(len(reacts[0]))]
        
    return react_sorted

In [34]:
def exctract_twitter_data(soup1):
    dictio = {}
    dictio['username']=twitter_user(soup1)
    dictio['date']=twitter_time(soup1)
    dictio['text']=twitter_tweets(soup1)
    react=twitter_react(soup1)
    dictio['replies']=react[0]
    dictio['reposts']=react[1]
    dictio['likes']=react[2]
    dictio['bookmarks']=react[3]
    dictio['views']=react[4]
    
    data = pd.DataFrame(dictio)
    
    return data

<h1> Scrapping with selenium

In [10]:
url = 'https://twitter.com/explore'
options = Options()
options.add_argument("--window-size=1920,1080")
tweets_df3=pd.DataFrame()


In [32]:
for i in range(6,12):    
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Wait for the login input field to be present
    wait = WebDriverWait(driver, 20)
    log = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@autocomplete='username']")))
    # Input the email
    log.send_keys('mana_isma@hotmail.com')
    log.send_keys(Keys.RETURN)

    # Input the pseudo
    pseudo = wait.until(EC.presence_of_element_located((By.XPATH, "//input")))
    pseudo.send_keys('Izumiu10')
    pseudo.send_keys(Keys.RETURN)


    # Input the password
    password=wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='password']")))
    password.send_keys('Izumiii0')
    password.send_keys(Keys.RETURN)


    #Search
    search = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@aria-label='Search query']")))
    search.send_keys('#palestine')

    search.send_keys(Keys.RETURN)



    search_parameter = wait.until(EC.presence_of_element_located((By.XPATH, "//button[@data-testid='searchBoxOverflowButton']")))
    search_parameter.click()

    search_advanced = wait.until(EC.presence_of_element_located((By.XPATH, "//a[@href='/search-advanced']")))
    search_advanced.click()


    search = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='allOfTheseWords']")))
    search.send_keys('#palestine')


    language = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_1']")))
    select_language = Select(language)
    select_language.select_by_value('en')

    #From_year = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_4']")))
    #From_day = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_3']")))
    #From_month = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_2']")))
    #select_fyear = Select(From_year)
    #select_fday = Select(From_day)
    #select_fmonth = Select(From_month)
    #select_fyear.select_by_value('2024')
    #select_fday.select_by_value('1')
    #select_fmonth.select_by_value('1')


    To_year = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_7']")))
    To_day = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_6']")))
    To_month = wait.until(EC.presence_of_element_located((By.XPATH, "//select[@id='SELECTOR_5']")))
    select_tyear = Select(To_year)
    select_tday = Select(To_day)
    select_tmonth = Select(To_month)
    select_tyear.select_by_value('2023')
    select_tday.select_by_value('1')
    select_tmonth.select_by_value(str(i))


    search = driver.find_element(By.XPATH, "//span[text()='Search']")
    search.click()




    time.sleep(40)
    SCROLL_PAUSE_TIME = 20

    itera = 0

    last_height = driver.execute_script("return document.body.scrollHeight")

    while itera <200:

        page_content = driver.page_source

        soup = BeautifulSoup(page_content, 'html.parser')

        try:
        # Code susceptible de lever une exception
            df = exctract_twitter_data(soup)
            tweets_df3 = pd.concat([tweets_df3, df], ignore_index=True)
        except TypeError as e:
            # Traitement de l'erreur TypeError
            print("Une erreur s'est produite :", e)
        except KeyError :
            print("Erreur de keyerror")

        itera+=1

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            itera+=50
        last_height = new_height


    page_content = driver.page_source

    soup = BeautifulSoup(page_content, 'html.parser')

    driver.quit()

/VictorNakba
/Bsonja5
/Bsonja5
/AntisemitismEye
/worldwarbot
/MuhammadSmiry
/Shanfaraa
/HananyaNaftali
/WebManar
/misbar_en
/jeremycorbyn
/tunahankuzu
/MuslimNetworkTV
/r1ndu19
/DrSabirplf
/georgegalloway
/PICA_Ambassador
/abierkhatib
/MrAndyNgo
/Act4pal1
/UNHumanRights
/RamzyBaroud
/Act4pal1
/EpshtainItay
/DailyNewsEgypt
/MayadeenEnglish
/georgegalloway
/georgegalloway
/SMQureshiPTI
/timand2037
/Lance63
/aviabelow
Une erreur s'est produite : 'NoneType' object is not subscriptable
/Isobel_waby
Une erreur s'est produite : 'NoneType' object is not subscriptable
/YousefAljamal
/UNHabitat_Ar
Une erreur s'est produite : 'NoneType' object is not subscriptable
/mariamtayob
Une erreur s'est produite : 'NoneType' object is not subscriptable
/MiddleEastMnt
/OrinocoTribune
/MiddleEastMnt
/MiddleEastMnt
/PalestinePDP
/OrinocoTribune
/MiddleEastMnt
/MiddleEastMnt
/PalestinePDP
/OrinocoTribune
/MiddleEastMnt
/MiddleEastMnt
/PalestinePDP
/OrinocoTribune
/MiddleEastMnt
/MiddleEastMnt
/PalestinePDP


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=125.0.6422.113); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CBAC1F52+60322]
	(No symbol) [0x00007FF7CBA3CEC9]
	(No symbol) [0x00007FF7CB8F7EBA]
	(No symbol) [0x00007FF7CB8FDCEE]
	(No symbol) [0x00007FF7CB900641]
	(No symbol) [0x00007FF7CB9006E0]
	(No symbol) [0x00007FF7CB94968C]
	(No symbol) [0x00007FF7CB93B346]
	(No symbol) [0x00007FF7CB96C21A]
	(No symbol) [0x00007FF7CB93ADB6]
	(No symbol) [0x00007FF7CB96C430]
	(No symbol) [0x00007FF7CB98BC80]
	(No symbol) [0x00007FF7CB96BFC3]
	(No symbol) [0x00007FF7CB939617]
	(No symbol) [0x00007FF7CB93A211]
	GetHandleVerifier [0x00007FF7CBDD94AD+3301629]
	GetHandleVerifier [0x00007FF7CBE236D3+3605283]
	GetHandleVerifier [0x00007FF7CBE19450+3563680]
	GetHandleVerifie

In [33]:
tweets_df3

,username,date,text,replies,reposts,likes,bookmarks,views
0,zalaahhhh,2024-05-31T08:35:45.000Z,Please don’t forget to pray for #Palestine,0,9,26,0,727
1,TIgerNS3,2024-05-29T19:43:28.000Z,"Dear world, wake up and unite against #IsIamic...",59,823,1569,75,156967
2,Chilliebeanz,2024-05-31T11:43:22.000Z,Look at how French police are treating their o...,0,12,19,0,231
3,SilentlySirs,2024-05-26T00:23:27.000Z,A little Palestinian girl arrived at the hospi...,9,71,75,5,2911
4,tinda_singh,2024-05-30T10:38:58.000Z,All Eyes on Rafah \n#RafafOnFıre #AllEyesOnRaf...,19,48,125,13,16077
...,...,...,...,...,...,...,...,...
854,PalestinePDP,2023-05-31T09:02:01.000Z,.@MiddleEastMnt \n\nWe are used to other count...,0,3,0,0,205
855,OrinocoTribune,2023-05-31T21:52:54.000Z,The Cultural Looting of Gaza\n\nhttps://orinoc...,0,0,0,0,116
856,MiddleEastMnt,2023-05-31T19:40:10.000Z,Watch: #Israeli soldier uses #Palestine flag t...,3,6,6,0,1608
857,MiddleEastMnt,2023-05-31T11:50:10.000Z,#Palestine: 13 years since #Israel's attack on...,3,9,8,0,1675


In [19]:
tweets_df = pd.read_csv('Palestine-Israel_tweets8.csv')

In [22]:
tweets_df = tweets_df.drop('Unnamed: 0',axis=1)

In [23]:
tweets_df

,username,date,text,replies,reposts,likes,bookmarks,views
0,CoreyBrickley,2022-12-12T03:30:54.000Z,just wanna save this for posterity in case i g...,337,10953,120306,3746,0
1,Scepticskeptic1,2022-03-17T01:54:42.000Z,EVERYONE SHUT UP THERES A NEW MINECRAFT WOOD TYPE,919,5806,81653,1038,0
2,CoreyBrickley,2022-12-15T17:41:51.000Z,idk why you’d tell on yourself like this,363,3412,63139,1019,5479623
3,CoreyBrickley,2022-12-12T03:30:54.000Z,just wanna save this for posterity in case i g...,337,10953,120306,3746,0
4,Scepticskeptic1,2022-03-17T01:54:42.000Z,EVERYONE SHUT UP THERES A NEW MINECRAFT WOOD TYPE,919,5806,81653,1038,0
...,...,...,...,...,...,...,...,...
3064,kkkKiran0,2024-05-30T15:54:21.000Z,"Der aaye durust aaye, finally Bollywood speaks...",3,11,18,0,407
3065,kkkKiran0,2024-05-30T03:20:37.000Z,I’m sorry for what you’re going through#RafafO...,3,30,58,2,5282
3066,war_noir,2024-05-25T20:16:23.000Z,#Lebanon / #Palestine / #Israel : #Hezbollah c...,0,17,63,8,10369
3067,SilentlySirs,2024-05-23T18:02:17.000Z,A picture of an unprecedented war crime where...,4,29,21,2,1715


In [24]:
tweets = pd.concat([tweets_df,tweets_df3],ignore_index=True)

In [28]:
tweets = tweets.drop_duplicates(subset=['text'])

In [29]:
tweets

,username,date,text,replies,reposts,likes,bookmarks,views
0,CoreyBrickley,2022-12-12T03:30:54.000Z,just wanna save this for posterity in case i g...,337,10953,120306,3746,0
1,Scepticskeptic1,2022-03-17T01:54:42.000Z,EVERYONE SHUT UP THERES A NEW MINECRAFT WOOD TYPE,919,5806,81653,1038,0
2,CoreyBrickley,2022-12-15T17:41:51.000Z,idk why you’d tell on yourself like this,363,3412,63139,1019,5479623
156,leeeebio,2022-07-17T17:14:33.000Z,Bro anime fans gotta chill on Sakura,337,2301,27015,476,0
157,galaxy_nette,2022-12-30T23:58:13.000Z,#MLBS5Spoilers\n\nTHE TIMELINE DROWNING IN ADR...,0,17,118,4,1938
...,...,...,...,...,...,...,...,...
3617,ifamericansknew,2024-01-31T22:45:20.000Z,Know the history. This was #palestinianlife un...,3,48,48,3,1265
3618,IranDailyWeb,2024-01-31T21:50:22.000Z,The #Frontpage of tomorrow's #Newspaper #Gaza ...,0,0,0,0,2333
3619,cgtnamerica,2024-01-31T22:21:04.000Z,Axios reports that U.S. Secretary of State Ant...,0,0,2,0,726
3620,ferozwala,2024-01-31T22:59:28.000Z,\nClashes with the Israeli occupation forces a...,0,0,2,0,123


In [26]:
tweets.to_csv("Palestine-Israel_tweets8.csv")

In [ ]:
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height